In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install rasterio

In [ ]:
import os
import glob
from sklearn import preprocessing
import rasterio
import numpy as np
from skimage.filters import threshold_otsu
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
scaler = preprocessing.MinMaxScaler()
output_path  = '/content/drive/MyDrive/Segmented_Images/'

In [ ]:
#Function apply mask to image
def apply_mask(image ,mask):
    masked_bands = []
    for band in image:
      idx=(mask==0)
      band[idx] = 0
      masked_bands.append(band)
    masked = np.array(masked_bands)
    return masked

# Function to read a band safely
def read_band(src, band_number):
    try:
        band = src.read(band_number).astype(float)
        return band
    except rasterio.errors.RasterioIOError as e:
        print(f"Error reading band {band_number}: {e}")
        return None

In [ ]:
def save_image(image, filename, profile):
    with rasterio.open(output_path + filename, 'w', **profile) as dst:
        dst.write(image)
        dst.descriptions = tuple(['B4', 'B3', 'B2', 'B1', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12'])

def open_files(folder_path):
    band5 = None
    band11 = None
    file_path = folder_path + '/*.tif'
    file_paths = sorted(glob.glob(file_path), key=lambda x: x.split('.')[0])
    for i in range(800, len(file_paths) - 1):
      file = file_paths[i]
      try:
        with rasterio.open(file) as src:
          # Read entire image
          image = src.read()
          # Get the profile of the source raster
          profile = src.profile

          # Read the required bands (B5 and B11 for SWI)
          band5 = read_band(src, 5)
          band11 = read_band(src, 11)
          band5 = scaler.fit_transform(band5)
          band11 = scaler.fit_transform(band11)

          if band5 is None or band11 is None:
             print("Failed to read necessary bands. Exiting.")
          else:
            # Calculate the SWI
            numerator = band5 - band11
            denominator = band5 + band11
            denominator[denominator == 0] = np.nan  # Avoid division by zero
            swi = numerator / denominator

            # Apply Otsu threshold to segment water bodies
            swi = np.nan_to_num(swi)  # Convert NaNs to 0
            threshold = threshold_otsu(swi)
            water_body = swi >= threshold

            # Apply mask to all 12 bands of image
            image = apply_mask(image, water_body)

            #Apply scaling to all bands
            image = np.transpose(image, (1,2,0))
            ascolumns = image.reshape(-1, 12)
            t = scaler.fit_transform(ascolumns)
            transformed = t.reshape(image.shape)
            image = np.transpose(transformed, (2,0,1))

            # Save segmented image
            filename = os.path.basename(file)
            save_image(image, filename, profile)
      except rasterio.errors.RasterioIOError as e:
        print(f"Error reading file {file_path}: {e}")
        continue


In [ ]:
folder_paths = ['/content/drive/MyDrive/Satellite_Image_Repo(1)',
                '/content/drive/MyDrive/Satellite_Image_Repo(2)',
                '/content/drive/MyDrive/Satellite_Image_Repo(3)']

open_files(folder_paths[1])
print("Finished with folder")